In [17]:
import pandas as pd
import nltk
import faiss
import torch
import re
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [18]:
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

# Obtener la clave de OpenAI desde el archivo .env
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Ahora puedes usar OPENAI_API_KEY para autenticarte con OpenAI
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm = ChatOpenAI(model="gpt-4o-mini")

In [19]:
def clean_text(text):
    text = text.lower()
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("\C","")
    text = re.sub(r"\[.*?\]", "", text) 
    return text.strip()

<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\dicam\AppData\Local\Temp\ipykernel_16552\583663975.py:3: SyntaxWarning: invalid escape sequence '\C'
  text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("\C","")


In [20]:
def process_txt_files(folder_path):
    df = pd.read_csv(folder_path)
    df["Entrevista"] = df["Entrevista"].apply(clean_text)  
    return df

In [21]:
folder_path = "Data\CorpusRI.csv"
df = process_txt_files(folder_path)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\dicam\AppData\Local\Temp\ipykernel_16552\2818103128.py:1: SyntaxWarning: invalid escape sequence '\C'
  folder_path = "Data\CorpusRI.csv"


In [22]:

text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=450,
                  chunk_overlap=0)




In [23]:
# Dividir los textos de las noticias
entrevistas = df["Entrevista"].tolist()
splits = []
for entrevista in entrevistas:
    splits += text_splitter.split_text(entrevista)


In [24]:

# Guardar los chunks en un DataFrame
chunks = [split for split in splits]
df_chunks = pd.DataFrame(chunks, columns=['Text'])


In [25]:
from langchain_openai import OpenAIEmbeddings
# Generación de embeddings
embeddings_model = OpenAIEmbeddings()


In [26]:
def embeddings_fn(text):
    """
    Crea los embeddings dado un texto
    """
    return embeddings_model.embed_query(text)


In [27]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Definir el modelo de embeddings
embeddings_model = OpenAIEmbeddings()

# # Calcular los embeddings
# df_chunks['Embeddings'] = df_chunks['Text'].apply(lambda x: embeddings_model.embed_query(x))

In [5]:
df_chunks.to_csv("EmbeddingsEntrevistas-gpt4.csv")

NameError: name 'df_chunks' is not defined

In [28]:
df_chunks = pd.read_csv("EmbeddingsEntrevistas.csv")

In [29]:
len(df_chunks['Embeddings'][0])

34339

In [30]:
import numpy as np
df_chunks['Embeddings'] = df_chunks['Embeddings'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))


In [46]:
import numpy as np

def find_best_passage(query, dataframe):
  """
  Calcula las distancias entre la consulta y cada documento en el marco de datos utilizando el producto punto.
  """
  query_embedding = embeddings_model.embed_query(query)

  # Revisa como opera la función dot aquí: https://numpy.org/doc/stable/reference/generated/numpy.dot.html
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding)

  # Obtiene la ID del que más se parece
  idx = np.argmax(dot_products)

  return dataframe.iloc[idx]['Text'] # Devuelve el texto del índice con el valor máximo.

In [55]:
query = "¿Que opina Andrea Gonzales sobre la corrupcion?"
best_passage = find_best_passage(query, df_chunks)
best_passage

'gobierno que representa los intereses los grupos de poder económico los grandes empresarios evasores de impuestos y que el pueblo ha dicho no voa nunca más qué opina de la candidatura de luisa gonzález eh el pasado de corrupción de asalto a los dineros del estado defensora y hoy eh cómplice también de los prófugo de la justicia además de recordarle que cuando fue ministra del trabajo dictó el el acuerdo 094 que pretendió eliminar la compensación'

In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


prompt = ChatPromptTemplate.from_messages([
    ("system", """
      Eres un asistente informativo que responde preguntas basándose en entrevistas con los candidatos presidenciales de Ecuador. 
     Proporciona respuestas claras y detalladas, explicando los temas de manera accesible para una audiencia no técnica. 
     Mantén un tono amigable y conversacional, desglosando conceptos complejos cuando sea necesario. Responde en el idioma en el que se haga la pregunta.

      PREGUNTA: '{query}'
      TEXTO DE REFERENCIA: '{relevant_passage}'

      RESPUESTA:
    """
    ),
])

chain = prompt | llm | output_parser

In [57]:
chain.invoke({
    "query": query,
    "relevant_passage": best_passage,
    })

'Andrea Gonzales ha expresado su opinión sobre la corrupción de manera clara y contundente. Ella critica fuertemente a los gobiernos que, según ella, están alineados con los intereses de grupos de poder económico y grandes empresarios que evaden impuestos. En este contexto, menciona que el pueblo ha mostrado un rechazo hacia estas prácticas y asegura que no desea volver a esos modelos de gobernanza.\n\nAdemás, se refiere a la candidatura de Luisa González y señala el pasado de corrupción que la rodea, incluyendo acusaciones sobre el manejo indebido de recursos públicos. Gonzales menciona que González fue defensora de aquellos que, según ella, han cometido actos de corrupción y también los califica de prófugos de la justicia. También critica un acuerdo que fue promulgado durante el tiempo en que González era ministra del trabajo, el cual según Gonzales, buscaba eliminar beneficios como la compensación laboral, lo que podría afectar a los trabajadores.\n\nEn resumen, Andrea Gonzales tien